In [ ]:
#!pip install matplotlib
#!pip install mapclassify

In [ ]:
import os
import folium
from SITS import Csv2gdf, StacAttack, Labels
import pandas as pd

In [ ]:
with open('var.txt', 'r') as f:
    input_tables = f.readlines()
    input_tables = [line.strip() for line in input_tables]

input_t1 = input_tables[0]
#input_t2 = input_tables[1]

In [ ]:
test01 = Csv2gdf(input_t1, 'long', 'Lat', 4326, 'Id')
#test02 = Csv2gdf(input_t2, 'longitude', 'latitude', 4326)
#test02.table = test02.table.drop(test02.table[(test02.table['coord_X'] == 'NaN')].index)# & (test02.table['coord_Y'] == 'NaN')].index)
#test02.table = test02.table[test02.table['coord_X'].notna() & test02.table['coord_Y'].notna()]

In [ ]:
test01.table.head()
type(test01.table)
print(isinstance(test01.table, pd.core.frame.DataFrame))

In [ ]:
out_dir = 'output01'
out_file = 'table01.geojson'

if not os.path.exists(out_dir):
    os.mkdir(out_dir)


test01.set_gdf(3035, os.path.join(out_dir, out_file))

In [ ]:
test01.set_buffer('gdf', 75)
test01.set_bbox('buffer')

In [ ]:
test01.to_vector('gdf', os.path.join(out_dir, 'gdf.geojson'))
test01.to_vector('buffer', os.path.join(out_dir, 'buffer.geojson'))
test01.to_vector('bbox', os.path.join(out_dir, 'bbox.geojson'))

In [ ]:
f = folium.Figure(height=400)
m = folium.Map(location=[55, 0], zoom_start=3).add_to(f)
test01.bbox.explore(m=m, height=400, color='green')
test01.buffer.explore(m=m, height=400)
test01.gdf.explore(m=m, height=400, color='red')

In [ ]:
for index, row in test01.bbox.head(1).iterrows():
    gid = test01.bbox.loc[index, 'gid']
    print("gid: ", gid)

    row_geom = test01.bbox.loc[index, 'geometry']
    row_geom_4326 = test01.bbox.to_crs(4326).loc[index, 'geometry']
    aoi_bounds = list(row_geom.bounds)
    print(aoi_bounds)
    aoi_bounds_4326 = list(row_geom_4326.bounds)

    imgs = StacAttack()
    imgs.searchItems(aoi_bounds_4326)

    imgs.loadPatches(aoi_bounds, 10, 10)
    print('nbbox', imgs.newbbox)
    imgs.loadImgs(aoi_bounds)
    print('obbox', imgs.newbbox)
    imgs.to_csv(out_dir, gid)
    imgs.to_nc('patch', gid, out_dir)
    imgs.to_nc('image', gid, out_dir)


In [ ]:
#type(test01.buffer)
#test = Labels(test01.buffer)
#test.to_raster(imgs.geobox, out_dir)

test = Labels("/home/osekenj/data/github_proj/SITS_utils/data/ajeter.shp")
test.to_raster("id", imgs.geobox, out_dir)
